In [22]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [23]:
class NeuralNetwork:
    def __init__(self, n_inputs, n_hidden, n_outputs):
        # Initialize the weights randomly
        self.weights1 = np.random.randn(n_inputs, n_hidden)
        self.weights2 = np.random.randn(n_hidden, n_outputs)

    def feedforward(self, X):
        # Calculate the output of the neural network for input X
        self.hidden = np.dot(X, self.weights1)
        self.hidden_activated = self.sigmoid(self.hidden)
        self.output = np.dot(self.hidden_activated, self.weights2)
        self.output_activated = self.sigmoid(self.output)
        return self.output_activated
    
    def sigmoid(self, x):
        # Sigmoid activation function
        return 1 / (1 + np.exp(-x))
    
    def softmax(self, x):
        exps = np.exp(x - np.max(x))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def forward(self, X):
        hidden = self.sigmoid(np.dot(X, self.weights1))
        output = self.softmax(np.dot(hidden, self.weights2))
        return output


In [32]:
# Define the particle swarm optimization class
class ParticleSwarmOptimization:
    def __init__(self, n_particles, n_inputs, n_hidden, n_outputs, max_iter, learning_rate):
        # Initialize the swarm randomly
        self.swarm = []
        for i in range(n_particles):
            neural_network = NeuralNetwork(n_inputs, n_hidden, n_outputs)
            self.swarm.append(neural_network)

        # Initialize the best position and error for each particle
        self.best_positions = []
        self.best_errors = []
        for i in range(n_particles):
            self.best_positions.append((self.swarm[i].weights1.copy(), self.swarm[i].weights2.copy()))
            self.best_errors.append(float("inf"))

        # Initialize the global best position and error
        self.global_best_position = None
        self.global_best_error = float("inf")

        # Initialize the parameters for the PSO algorithm
        self.max_iter = max_iter
        self.learning_rate = learning_rate

    def calculate_error(self, X, y, neural_network):
        y_pred = np.argmax(neural_network.forward(X), axis=1)
        return np.mean(y_pred != y)

    def optimize(self, X, y):
        velocity1 = np.zeros_like(self.swarm[0].weights1)
        velocity2 = np.zeros_like(self.swarm[0].weights2)
        for i in range(self.max_iter):
            for j in range(len(self.swarm)):
                # Calculate the error for the current particle
                error = self.calculate_error(X, y, self.swarm[j])

                # Update the best position and error for the current particle
                if error < self.best_errors[j]:
                    self.best_positions[j] = (self.swarm[j].weights1.copy(), self.swarm[j].weights2.copy())
                    self.best_errors[j] = error

                # Update the global best position and error
                if error < self.global_best_error:
                    self.global_best_position = (self.swarm[j].weights1.copy(), self.swarm[j].weights2.copy())
                    self.global_best_error = error

                # Update the velocity and position for the current particle
                r1 = np.random.rand(*self.swarm[j].weights1.shape)
                r2 = np.random.rand(*self.swarm[j].weights2.shape)
                velocity1 = self.learning_rate * (velocity1 + r1 * (self.best_positions[j][0] - self.swarm[j].weights1) + r2 * (self.global_best_position[0] - self.swarm[j].weights1))
                velocity2 = self.learning_rate * (velocity2 + r1 * (self.best_positions[j][1] - self.swarm[j].weights2) + r2 * (self.global_best_position[1] - self.swarm[j].weights2))
                self.swarm[j].weights



In [33]:
data=pd.read_csv(r'C:\Users\anjan\Downloads\P16-Artificial-Neural-Networks\Part 1 - Artificial Neural Networks\Churn_Modelling.csv')

In [34]:
X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [35]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [36]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [38]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
n_particles = 10
n_inputs = X_train.shape[1]
n_hidden = 5
n_outputs = len(np.unique(y_train))
max_iter = 50
learning_rate = 0.5
pso = ParticleSwarmOptimization(n_particles, n_inputs, n_hidden, n_outputs, max_iter, learning_rate)

# Train the neural network using PSO
pso.optimize(X_train, y_train)

# Evaluate the performance of the neural network on the testing set
y_pred = np.argmax(pso.global_best_position[1] @ pso.global_best_position[0].T @ X_test.T, axis=0)
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)